In [52]:
from pyDeepInsight import ImageTransformer
from pyDeepInsight.utils import Norm2Scaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
from umap import UMAP
import os

In [53]:
df = pd.read_csv("dataset.csv")
df.head()

,ClassLabel,X01,Y01,Z01,X11,Y11,Z11,X21,Y21,Z21,...,Z6219,X6319,Y6319,Z6319,X6419,Y6419,Z6419,X6519,Y6519,Z6519
0,Boy,1.0,0.547994,0.000000,1.0,0.547994,0.000000,1.0,0.547994,0.000000,...,0.071914,0.272580,0.322938,0.127316,0.228388,0.264486,0.084617,0.265310,0.313749,0.177373
1,Boy,1.0,0.857104,0.379246,1.0,0.857104,0.379246,1.0,0.857104,0.379246,...,0.265342,0.458347,0.407922,0.230979,0.368018,0.479731,0.278366,0.415877,0.466096,0.319931
2,Boy,1.0,0.852530,0.358460,1.0,0.852530,0.358460,1.0,0.852530,0.358460,...,0.252582,0.462702,0.409546,0.237752,0.368986,0.483866,0.267000,0.418716,0.471061,0.319184
3,Boy,1.0,0.778636,0.000000,1.0,0.778636,0.000000,1.0,0.778636,0.000000,...,0.003979,0.426772,0.485623,0.153090,0.321029,0.400402,0.032188,0.410688,0.467018,0.270999
4,Boy,1.0,0.589883,0.119620,1.0,0.589883,0.119620,1.0,0.589883,0.119620,...,0.380987,0.487479,0.526593,0.386855,0.465436,0.526274,0.390475,0.482003,0.520124,0.418845


In [54]:
y = df['ClassLabel'].values
X = df.iloc[:, 1:].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y)

In [67]:
num_nan = df.isna().sum()
num_nan

ClassLabel      0
X01             0
Y01             0
Z01             0
X11             0
             ... 
Y6419         300
Z6419         300
X6519         300
Y6519         300
Z6519         300
Length: 3763, dtype: int64

In [62]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

le_mapping = dict(zip(le.transform(le.classes_), le.classes_))
num_classes = np.unique(y_train_enc).size

In [57]:
print("Original X_train size:", X_train.size)
print("Expected size after reshaping:", 577 * 198 * 19 * 1)

# Add this line to check the dimensions of X_train before reshaping
print("X_train dimensions before reshaping:", X_train.shape)

Original X_train size: 2170674
Expected size after reshaping: 2170674
X_train dimensions before reshaping: (577, 3762)


In [58]:
# Model Traiining using CNN

import numpy as np
import pickle
import cv2, os
from glob import glob
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K


# K.set_image_data_format('channels_last')

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# def get_num_of_classes():
# 	return len(np.unique(y, return_counts=True)[0])

# coords = 198
# frames = 19

# y_train_enc = to_categorical(y_train_enc, num_classes=17)
# y_test_enc = to_categorical(y_test_enc, num_classes=17)

# def cnn_model():
#     num_of_classes = get_num_of_classes()
#     model = Sequential()
#     # Corrected input shape: (height, width, channels)
#     model.add(Conv2D(16, (2,2), input_shape=(198, 19, 1), strides=(1, 1), padding='SAME', data_format='channels_last'))
#     model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
#     model.add(Conv2D(32, (3,3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
#     model.add(Conv2D(64, (1, 1), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
#     model.add(Flatten())
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(num_of_classes, activation='softmax'))
#     sgd = SGD(lr=1e-2)
#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#     filepath="cnn_model_keras2.h5"
#     checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#     callbacks_list = [checkpoint1]
#     return model, callbacks_list

#     return model, callbacks_list
# def train(X_train, X_test, y_train_enc, y_test_enc):
# 	X_train = X_train.reshape(X_train.shape[0],coords, frames, 1).astype('float32')
# 	X_test = X_test.reshape(X_test.shape[0], coords, frames, 1).astype('float32')

# 	model, callbacks_list = cnn_model()
# 	model.summary()
# 	model.fit(X_train, y_train_enc, validation_data=(X_test, y_test_enc), epochs=15, batch_size=500, callbacks=callbacks_list)
# 	scores = model.evaluate(X_test, y_test_enc, verbose=0)
# 	print("CNN Error: %.2f%%" % (100-scores[1]*100))
# 	#model.save('cnn_model_keras2.h5')

# train(X_train, X_test, y_train_enc, y_test_enc)
# K.clear_session();


In [61]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train[: , :])
X_test_norm = mms.transform(X_test[:, :])




(3762,)

In [63]:
import umap

reducer = umap.UMAP(
    n_components=2,
    #min_dist=0.8,
    metric='cosine',
    n_jobs=-1
)

pixel_size = (224,224)
it = ImageTransformer(
    feature_extractor=reducer,
    pixels=pixel_size)

In [64]:
it.fit(X_train_norm, y=y_train, plot=True)
X_train_img = it.transform(X_train_norm)
X_test_img = it.transform(X_test_norm)

ValueError: Input contains NaN.